# Introduction to Cross-Validation - Lab

## Introduction

In this lab, you'll be able to practice your cross-validation skills!


## Objectives

You will be able to:

- Compare the results with normal holdout validation
- Apply 5-fold cross validation for regression

## Let's get started

This time, let's only include the variables that were previously selected using recursive feature elimination. We included the code to preprocess below.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.datasets import load_boston

boston = load_boston()

boston_features = pd.DataFrame(boston.data, columns = boston.feature_names)
b = boston_features["B"]
logdis = np.log(boston_features["DIS"])
loglstat = np.log(boston_features["LSTAT"])

# minmax scaling
boston_features["B"] = (b-min(b))/(max(b)-min(b))
boston_features["DIS"] = (logdis-min(logdis))/(max(logdis)-min(logdis))

#standardization
boston_features["LSTAT"] = (loglstat-np.mean(loglstat))/np.sqrt(np.var(loglstat))

In [3]:
X = boston_features[['B','DIS','LSTAT']]
y = pd.DataFrame(boston['target'],columns=['MEDV'])

## Train test split

Perform a train-test-split with a test set of 0.20.

In [4]:
from sklearn.model_selection import train_test_split

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)

In [6]:
print(len(X_train), len(X_test), len(y_train), len(y_test))

404 102 404 102


Fit the model and apply the model to the make test set predictions

In [7]:
from sklearn.linear_model import LinearRegression
linreg = LinearRegression()
linreg.fit(X_train, y_train)

y_hat_train = linreg.predict(X_train)
y_hat_test = linreg.predict(X_test)

Calculate the residuals and the mean squared error

In [8]:
train_residuals = y_hat_train - y_train
test_residuals = y_hat_test - y_test

mse_train = np.sum((y_train-y_hat_train)**2)/len(y_train)
mse_test =np.sum((y_test-y_hat_test)**2)/len(y_test)
print('Train Mean Squarred Error:', mse_train)
print('Test Mean Squarred Error:', mse_test)

Train Mean Squarred Error: MEDV    26.069163
dtype: float64
Test Mean Squarred Error: MEDV    23.323436
dtype: float64


## Cross-Validation: let's build it from scratch!

### Create a cross-validation function

Write a function k-folds that splits a dataset into k evenly sized pieces.
If the full dataset is not divisible by k, make the first few folds one larger then later ones.

We want the folds to be a list of subsets of data!

In [9]:
import math
def kfolds(data, k):
    # Force data as pandas dataframe
    df = pd.DataFrame(data)
    # add 1 to fold size to account for leftovers    
    section = math.ceil(len(data)/k)
    data_sections = []
    for i in range(k):
        x = i+1
        start = i * section
        stop = x * section
        data_sections.append(data.iloc[start:stop])
    return data_sections

### Apply it to the Boston Housing Data

In [10]:
# Make sure to concatenate the data again
kset = X.join(y)

In [11]:
kset = kfolds(kset,5)

### Perform a linear regression for each fold, and calculate the training and test error

Perform linear regression on each and calculate the training and test error.

In [12]:
test_errs = []
train_errs = []
k=5

for n in range(k):
    a = kset.copy()
    # Split in train and test for the fold
    Xa_test = kset[n][['B','DIS','LSTAT']]
    ya_test = kset[n]['MEDV']
    
    a.pop(n)
    Xa_train = pd.concat(a)[['B','DIS','LSTAT']]
    ya_train = pd.concat(a)['MEDV']
    # Fit a linear regression model
    linreg.fit(Xa_train, ya_train)

    ya_hat_train = linreg.predict(Xa_train)
    ya_hat_test = linreg.predict(Xa_test)
    #Evaluate Train and Test Errors
    atrain_residuals = ya_hat_train - ya_train
    atest_residuals = ya_hat_test - ya_test

    amse_train = np.sum((ya_train-ya_hat_train)**2)/len(ya_train)
    amse_test =np.sum((ya_test-ya_hat_test)**2)/len(ya_test)
    train_errs.append(amse_train)
    test_errs.append(amse_test)
    
print(train_errs)
print(test_errs)

[27.416804071996566, 26.721896026975926, 20.829507979031753, 22.925094249806698, 27.67352489975728]
[19.514663673647416, 21.19965996548275, 47.52812453809723, 38.745059033502464, 16.962593987622505]


## Cross-Validation using Scikit-Learn

This was a bit of work! Now, let's perform 5-fold cross-validation to get the mean squared error through scikit-learn. Let's have a look at the five individual MSEs and explain what's going on.

In [13]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

cv_5_results = np.mean(cross_val_score(linreg, X, y, cv=5, scoring="neg_mean_squared_error"))

print(cv_5_results)

-28.737736096449662


Next, calculate the mean of the MSE over the 5 cross-validations and compare and contrast with the result from the train-test-split case.

In [14]:
print(np.mean(train_errs))
print(np.mean(test_errs))

25.113365445513644
28.790020239670476


Using train_test_split yields varying results. Multiple iterations appear the value is favoring close to the mean of the MSE in the 5 cross-validations

##  Summary 

Congratulations! You now practiced your knowledge on k-fold crossvalidation!